# SetUp

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 31.2 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 18.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp39-cp39-linux_x86_64.whl size=680529 sha256=0008db30283a7d7b6971c8740871e5a1d9e69ec482f35dcb33a25cc7aa573b2f
  Stored in directory: /root/.cache/pip/wheels/47/17/70/b257bc53879a458c4bfcc900e89271aa8b4f19366a54bd2455
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-yjcec8ft
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-yjcec8ft
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 102.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.9/881.9 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 94.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install gensim==3.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 68.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gensim: filename=gensim-3.6.0-cp39-cp39-linux_x86_64.whl size=23915439 sha256=3dfcf82483854494f9b2cfc11f416c0bd82bd5ce55213dc1ba18f54254fadba7
  Stored in directory: /root/.cache/pip/wheels/61/12/f2/84de20fba5e870553796b0834d11109992f06ddc20aaead086
Successfully built gensim
  Attempting uninstall: gensim
    Found existing installation: gensim 4.3.1
    Uninstalling gensim-4.3.1:
      Successfully uninstalled gensim-4.3.1


# Model Load

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
import pandas as pd

In [5]:
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [6]:
import torch
from torch import nn
import torch.nn.functional as F

In [7]:
fpath = 'drive/MyDrive/tripbuilder_data/TLD/'

In [8]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))
        
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=1202,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [9]:
device = torch.device("cuda:0")
device

device(type='cuda', index=0)

In [10]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [14]:
model = torch.load('drive/MyDrive/tripbuilder_modules/TRIFIT/BertClassifier_model')

# Predict

In [15]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [16]:
fpath = 'drive/MyDrive/tripbuilder_data/TLD/'

encoder = LabelEncoder()
data_use = pd.read_csv(fpath+'Data_for_Encoder.csv')
encoder.fit(list(data_use['class']))

encoded_words = encoder.transform(data_use["class"])
word_dict = dict(zip(range(len(encoder.classes_)), encoder.classes_))

In [17]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [18]:
max_len = 30
batch_size = 16
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [64]:
def predict(predict_sentences):

    data = [[predict_sentence, '0'] for predict_sentence in predict_sentences]
    another_test = BERTDataset(data, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    result = []

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for logits in out:
            logit = logits.detach().cpu().numpy()
            res_label = encoder.inverse_transform([np.argmax(logit)])[0]
            result.append(res_label)

    return result

In [37]:
import warnings
warnings.filterwarnings('ignore')

In [156]:
fpath = 'drive/MyDrive/tripbuilder_data/PLA000/'
data = pd.read_json(fpath+'Seoul_PLA000.json')

data.head()

,name,id,cate1,cate2,locX,locY,address,callnum,time,restday,contents,imgs,tag,num_txt,staytime,cong
0,마곡장,SEO_0,음식점,음식점,126.829238,37.558123,"서울특별시 강서구 마곡중앙4로 22, 파인스퀘어 B동 1층 110호 (마곡동)",None,NaN,NaN,[안녕하세요~! 경제적 자유를 위해 공부하고 행동하는 플랙비입니당!​이번에는 서울 ...,NaN,NaN,99375,NaN,NaN
1,더바,SEO_1,음식점,음식점,127.027916,37.493962,"서울특별시 강남구 삼성로86길 11, 거봉INC 지하1층 (대치동)",None,NaN,NaN,[털삐예요❣​텍사스데브라질 갔다가 들르면 딱 좋은 거리압구정역에서 걸어서 7분정도 ...,NaN,NaN,99037,NaN,NaN
2,사이커피바,SEO_2,카페,음식점,127.033280,37.663038,"서울특별시 도봉구 도당로9길 22, 1층 (방학동)",None,NaN,NaN,[ 도봉구 방학동에 위치한 동네카페 ‘사이커피바(Sighcoffeebar)’에 다녀...,NaN,NaN,98680,NaN,NaN
3,종로맛집,SEO_3,음식점,"육류,고기",126.994503,37.570158,"서울특별시 종로구 통일로 156-3, 1층 (교남동)",02-2263-6658,NaN,NaN,[​한 주의 절반이 지나갔네요!!이번 주말에는 뭐 드실 계획이신가용???저는 지난 ...,NaN,NaN,98945,NaN,NaN
4,하루,SEO_4,음식점,음식점,127.055578,37.507680,"서울특별시 강남구 언주로30길 57, 지하1층 (도곡동, 타워팰리스)",02-538-4555,NaN,NaN,[서울 강남구을 당원협의회에 방문했습니다.궂은 날씨에도 많은 당원 여러분이 모여주셨...,NaN,NaN,96690,NaN,NaN


In [157]:
import time

def getTime(sec):
  sec = int(sec)
  h = sec//3600
  m = (sec-h*3600)//60
  s = (sec-h*3600)%60
  return f"{h}h {m}m {s}s"

In [158]:
data = data.iloc[5000:]
data.reset_index(drop=True,inplace=True)

In [159]:
cont_data = data["contents"]

In [160]:
import re

def process(text):
    text = re.sub(r'\u200b', '.', text)
    text = re.sub(r'[\!\~\?\^\n\/:]', '.', text)
    text = re.sub(r'[\u3131-\u314E\u314F-\u3163]', '', text)
    text = re.sub(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF]', '', text)
    text = re.sub(r'o', '', text)

    return text

In [161]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="gensim.summarization.summarizer")

In [ ]:
"""
import gensim.summarization as gs

class_lst = []

s = time.time()
for i in range(len(cont_data)):
  curr_cont = []
  for j in range(len(cont_data[i])):    
    sum_cont = gs.summarize(cont_data[i][j]+". "+"안녕하세요. "*5)
    for sent in sum_cont.split("."):
      sent = process(sent)
      if len(sent)==0:
        continue
      sent = "".join(sent.split(" "))
      curr_cont.append(predict(sent))
      t = time.time()-s
    print(f"\r[{j+1}/{len(cont_data[i])}]|Completed...[{getTime(t/(j+1)*(len(cont_data[i])-j-1))}]Remain",end="")
  class_lst.append(curr_cont)
  break
e = time.time()
print(f"times:{e-s} sec")
print(len(curr_cont))
"""

'\nimport gensim.summarization as gs\n\nclass_lst = []\n\ns = time.time()\nfor i in range(len(cont_data)):\n  curr_cont = []\n  for j in range(len(cont_data[i])):    \n    sum_cont = gs.summarize(cont_data[i][j]+". "+"안녕하세요. "*5)\n    for sent in sum_cont.split("."):\n      sent = process(sent)\n      if len(sent)==0:\n        continue\n      sent = "".join(sent.split(" "))\n      curr_cont.append(predict(sent))\n      t = time.time()-s\n    print(f"\r[{j+1}/{len(cont_data[i])}]|Completed...[{getTime(t/(j+1)*(len(cont_data[i])-j-1))}]Remain",end="")\n  class_lst.append(curr_cont)\n  break\ne = time.time()\nprint(f"times:{e-s} sec")\nprint(len(curr_cont))\n'

In [ ]:
import gensim.summarization as gs
from IPython.display import clear_output

spam_len = 9500; class_lst = []

s = time.time()
for i in range(len(cont_data)):
  data_lst =  [". ".join(process(cdata).split(".")) for cdata in cont_data[i] if len(cdata)<spam_len]
  sum_lst = []

  for j in range(0,len(data_lst),3):
    try:
      sum_cont = gs.summarize(". ".join(data_lst[j:j+3]),ratio=0.1,split=True)
      sum_lst+=sum_cont
  
    except:
      pass
  class_lst.append(predict(sum_lst))
  t = time.time()-s
  print(f"\r[{i+1}/{len(cont_data)}]|[{len(predict(sum_lst))}]Completed...[{getTime(t/(i+1)*(len(cont_data)-i-1))}]Remain",end="")

  if i%100==99:
    clear_output()

e = time.time()
print(f"times:{e-s} sec")

fpath = 'drive/MyDrive/tripbuilder_data/PLA003/'

data["tag"] = class_lst
data[['name', 'id', 'cate1', 'tag']].to_json(fpath+"Seoul_PLA003_6.json")

[18/984]|[74]Completed...[0h 28m 46s]Remain

In [154]:
"""
import gensim.summarization as gs
from IPython.display import clear_output

class_lst = []

s = time.time()
for i in range(len(cont_data)):
  curr_cont = []
  for j in range(len(cont_data[i])):    
    sum_cont = gs.summarize(cont_data[i][j]+". "+"안녕하세요. "*5,word_count=40)
    for sent in sum_cont.split("."):
      sent = process(sent)
      if len(sent)==0:
        continue
      #sent = " ".join(sent.split(" "))
      curr_cont.append(predict(sent))
      t = time.time()-s
  clear_output()
  print(f"\r[{i+1}/{len(cont_data)}]|Completed...[{getTime(t/(i+1)*(len(cont_data)-i-1))}]Remain",end="")
  class_lst.append(curr_cont)

e = time.time()
print(f"times:{e-s} sec")
print(len(curr_cont))

fpath = 'drive/MyDrive/tripbuilder_data/PLA003/'

data["tag"] = class_lst
data[['name', 'id', 'cate1', 'tag']].to_json(fpath+"Seoul_PLA003_1.json")
"""

'\nimport gensim.summarization as gs\nfrom IPython.display import clear_output\n\nclass_lst = []\n\ns = time.time()\nfor i in range(len(cont_data)):\n  curr_cont = []\n  for j in range(len(cont_data[i])):    \n    sum_cont = gs.summarize(cont_data[i][j]+". "+"안녕하세요. "*5,word_count=40)\n    for sent in sum_cont.split("."):\n      sent = process(sent)\n      if len(sent)==0:\n        continue\n      #sent = " ".join(sent.split(" "))\n      curr_cont.append(predict(sent))\n      t = time.time()-s\n  clear_output()\n  print(f"\r[{i+1}/{len(cont_data)}]|Completed...[{getTime(t/(i+1)*(len(cont_data)-i-1))}]Remain",end="")\n  class_lst.append(curr_cont)\n\ne = time.time()\nprint(f"times:{e-s} sec")\nprint(len(curr_cont))\n\nfpath = \'drive/MyDrive/tripbuilder_data/PLA003/\'\n\ndata["tag"] = class_lst\ndata[[\'name\', \'id\', \'cate1\', \'tag\']].to_json(fpath+"Seoul_PLA003_1.json")\n'

In [155]:
"""# Testing Code: 약 25초 소요(장소 1개, 글 40개 처리)
for j in range(len(cont_data[i])):    
    sum_cont = gs.summarize(cont_data[i][j]+". "+"안녕하세요. "*5,word_count=40)
    for sent in sum_cont.split("."):
      sent = process(sent)
      if len(sent)==0:
        continue
      sent = " ".join(sent.split(" "))
      curr_cont.append(predict(sent))"""

'# Testing Code: 약 25초 소요(장소 1개, 글 40개 처리)\nfor j in range(len(cont_data[i])):    \n    sum_cont = gs.summarize(cont_data[i][j]+". "+"안녕하세요. "*5,word_count=40)\n    for sent in sum_cont.split("."):\n      sent = process(sent)\n      if len(sent)==0:\n        continue\n      sent = " ".join(sent.split(" "))\n      curr_cont.append(predict(sent))'